In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.6`

import $ivy.$

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Files")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/10 13:48:12 INFO SparkContext: Running Spark version 3.5.6
25/12/10 13:48:12 INFO SparkContext: OS info Mac OS X, 26.1, aarch64
25/12/10 13:48:12 INFO SparkContext: Java version 11.0.29
25/12/10 13:48:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


spark: SparkSession = org.apache.spark.sql.SparkSession@308985f3
import spark.implicits._

In [5]:
println(s"spark.version == ${spark.version}")

spark.version == 3.5.6


## CSV

In [6]:
val path = "/opt/spark/examples/src/main/resources/people.csv"
val df = spark.read.csv(path)
df.show()

+------------------+
|               _c0|
+------------------+
|      name;age;job|
|Jorge;30;Developer|
|  Bob;32;Developer|
+------------------+



path: String = "/opt/spark/examples/src/main/resources/people.csv"
df: DataFrame = [_c0: string]

In [7]:
val df2 = spark.read.option("delimiter", ";").csv(path)
df2.show()

+-----+---+---------+
|  _c0|_c1|      _c2|
+-----+---+---------+
| name|age|      job|
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



df2: DataFrame = [_c0: string, _c1: string ... 1 more field]

In [8]:
val df3 = spark.read.option("delimiter", ";").option("header", "true").csv(path)
df3.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



df3: DataFrame = [name: string, age: string ... 1 more field]

In [9]:
val df4 = spark.read.options(Map("delimiter"->";", "header"->"true")).csv(path)

df4: DataFrame = [name: string, age: string ... 1 more field]

In [10]:
val booksPath = "data/books.csv"
val books = spark.read.option("header", "true").csv("data/books.csv")
books.show(5, false)

+------------------------------------------------------------------+------------------------+-----------+-------+-----+----+-----------+
|Name                                                              |Author                  |User Rating|Reviews|Price|Year|Genre      |
+------------------------------------------------------------------+------------------------+-----------+-------+-----+----+-----------+
|10-Day Green Smoothie Cleanse                                     |JJ Smith                |4.7        |17350  |8    |2016|Non Fiction|
|11/22/63: A Novel                                                 |Stephen King            |4.6        |2052   |22   |2011|Fiction    |
|12 Rules for Life: An Antidote to Chaos                           |Jordan B. Peterson      |4.7        |18979  |15   |2018|Non Fiction|
|1984 (Signet Classics)                                            |George Orwell           |4.7        |21424  |6    |2017|Fiction    |
|5,000 Awesome Facts (About Everything!) 

booksPath: String = "data/books.csv"
books: DataFrame = [Name: string, Author: string ... 5 more fields]

In [11]:
books.printSchema

root
 |-- Name: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- User Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)



In [12]:
val booksSchema = StructType(
      StructField("Name", StringType, nullable = true) ::
        StructField("Author", StringType, nullable = true) ::
        StructField("User Rating", DoubleType, nullable = true) ::
        StructField("Reviews", IntegerType, nullable = true) ::
        StructField("Price", IntegerType, nullable = true) ::
        StructField("Year", IntegerType, nullable = true) ::
        StructField("Genre", StringType, nullable = true) :: Nil
    )

booksSchema: StructType = StructType(
  StructField("Name", StringType, true, {}),
  StructField("Author", StringType, true, {}),
  StructField("User Rating", DoubleType, true, {}),
  StructField("Reviews", IntegerType, true, {}),
  StructField("Price", IntegerType, true, {}),
  StructField("Year", IntegerType, true, {}),
  StructField("Genre", StringType, true, {})
)

In [13]:
val booksSchemaDF = spark.read.option("header", "true").schema(booksSchema).csv("data/books.csv")
booksSchemaDF.printSchema

root
 |-- Name: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- User Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)



booksSchemaDF: DataFrame = [Name: string, Author: string ... 5 more fields]

In [14]:
booksSchemaDF.show(5, false)

+------------------------------------------------------------------+------------------------+-----------+-------+-----+----+-----------+
|Name                                                              |Author                  |User Rating|Reviews|Price|Year|Genre      |
+------------------------------------------------------------------+------------------------+-----------+-------+-----+----+-----------+
|10-Day Green Smoothie Cleanse                                     |JJ Smith                |4.7        |17350  |8    |2016|Non Fiction|
|11/22/63: A Novel                                                 |Stephen King            |4.6        |2052   |22   |2011|Fiction    |
|12 Rules for Life: An Antidote to Chaos                           |Jordan B. Peterson      |4.7        |18979  |15   |2018|Non Fiction|
|1984 (Signet Classics)                                            |George Orwell           |4.7        |21424  |6    |2017|Fiction    |
|5,000 Awesome Facts (About Everything!) 

## JSON

In [15]:
val customer = spark.read.json("data/customer.json")
customer.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)



customer: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [16]:
customer.show(5, false)

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

In [17]:
val countries = spark
                .read
                .format("json")
                .option("multiLine", "true")
                .load("data/countries.json")

25/12/10 13:48:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


countries: DataFrame = [altSpellings: array<string>, area: double ... 21 more fields]

In [18]:
countries.show(5, false)

+-----------------------------------------------+---------+------------------------------+------------+----+----+----+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
countries.printSchema

root
 |-- altSpellings: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area: double (nullable = true)
 |-- borders: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- capital: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- ccn3: string (nullable = true)
 |-- cioc: string (nullable = true)
 |-- currencies: struct (nullable = true)
 |    |-- AED: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- symbol: string (nullable = true)
 |    |-- AFN: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- symbol: string (nullable = true)
 |    |-- ALL: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- symbol: string (nullable = true)
 |    |-- AMD: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- symbol:

## Parquet

In [20]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4ca08c4b

In [21]:
val squaresDF = sc.makeRDD(1 to 5).map(i => (i, i * i)).toDF("value", "square")
squaresDF.write.mode(SaveMode.Overwrite).parquet("data/test_table/key=1")

squaresDF: DataFrame = [value: int, square: int]

In [22]:
val cubesDF = spark.sparkContext.makeRDD(6 to 10).map(i => (i, i * i * i)).toDF("value", "cube")
cubesDF.write.mode(SaveMode.Overwrite).parquet("data/test_table/key=2")

cubesDF: DataFrame = [value: int, cube: int]

In [23]:
val mergedDF = spark.read.option("mergeSchema", "true").parquet("data/test_table")
mergedDF.printSchema()

root
 |-- value: integer (nullable = true)
 |-- square: integer (nullable = true)
 |-- cube: integer (nullable = true)
 |-- key: integer (nullable = true)



mergedDF: DataFrame = [value: int, square: int ... 2 more fields]

In [24]:
mergedDF.show

+-----+------+----+---+
|value|square|cube|key|
+-----+------+----+---+
|    1|     1|NULL|  1|
|    4|    16|NULL|  1|
|    5|    25|NULL|  1|
|    2|     4|NULL|  1|
|    3|     9|NULL|  1|
|    7|  NULL| 343|  2|
|    8|  NULL| 512|  2|
|    6|  NULL| 216|  2|
|    9|  NULL| 729|  2|
|   10|  NULL|1000|  2|
+-----+------+----+---+



In [25]:
mergedDF.write.mode(SaveMode.Overwrite).parquet("data/merged.parquet")

In [26]:
import scala.sys.process._

"parquet-tools show data/merged.parquet".!!

import scala.sys.process._
res26_1: String = """+---------+----------+--------+-------+
|   value |   square |   cube |   key |
|---------+----------+--------+-------|
|       1 |        1 |    nan |     1 |
|       4 |       16 |    nan |     1 |
|       5 |       25 |    nan |     1 |
|       2 |        4 |    nan |     1 |
|       3 |        9 |    nan |     1 |
|       7 |      nan |    343 |     2 |
|       8 |      nan |    512 |     2 |
|       6 |      nan |    216 |     2 |
|       9 |      nan |    729 |     2 |
|      10 |      nan |   1000 |     2 |
+---------+----------+--------+-------+
"""

In [27]:
"parquet-tools inspect data/merged.parquet".!!

Traceback (most recent call last):
  File "/opt/homebrew/bin/parquet-tools", line 8, in <module>
    sys.exit(main())
             ~~~~^^
  File "/opt/homebrew/lib/python3.13/site-packages/parquet_tools/cli.py", line 26, in main
    args.handler(args)
    ~~~~~~~~~~~~^^^^^^
  File "/opt/homebrew/lib/python3.13/site-packages/parquet_tools/commands/inspect.py", line 55, in _cli
    _execute_simple(
    ~~~~~~~~~~~~~~~^
        filename=local_path,
        ^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/opt/homebrew/lib/python3.13/site-packages/parquet_tools/commands/inspect.py", line 63, in _execute_simple
    pq_file: pq.ParquetFile = pq.ParquetFile(filename)
                              ~~~~~~~~~~~~~~^^^^^^^^^^
  File "/opt/homebrew/lib/python3.13/site-packages/pyarrow/parquet/core.py", line 324, in __init__
    source = filesystem.open_input_file(source)
  File "pyarrow/_fs.pyx", line 815, in pyarrow._fs.FileSystem.open_input_file
  File "pyarrow/error.pxi", line 155, in pyarrow.lib.pyarro

java.lang.RuntimeException: Nonzero exit value: 1

In [28]:
"ls -1 data/merged.parquet".!!

res28: String = """_SUCCESS
part-00000-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet
part-00001-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet
part-00002-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet
part-00003-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet
part-00004-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet
"""

In [29]:
val files = "ls -1 data/merged.parquet".!!.split("\n").filter(_ != "_SUCCESS")

files: Array[String] = Array(
  "part-00000-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet",
  "part-00001-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet",
  "part-00002-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet",
  "part-00003-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet",
  "part-00004-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet"
)

In [30]:
val file1 = files(0)

s"parquet-tools inspect data/merged.parquet/$file1".!!

file1: String = "part-00000-4aa1e984-d12e-4e67-8254-fc9cb7855259-c000.snappy.parquet"
res30_1: String = """
############ file meta data ############
created_by: parquet-mr version 1.13.1 (build db4183109d5b734ec5930d870cdae161e408ddba)
num_columns: 4
num_rows: 2
num_row_groups: 1
format_version: 1.0
serialized_size: 838


############ Columns ############
value
square
cube
key

############ Column(value) ############
name: value
path: value
max_definition_level: 1
max_repetition_level: 0
physical_type: INT32
logical_type: None
converted_type (legacy): NONE
compression: SNAPPY (space_saved: -6%)

############ Column(square) ############
name: square
path: square
max_definition_level: 1
max_repetition_level: 0
physical_type: INT32
logical_type: None
converted_type (legacy): NONE
compression: SNAPPY (space_saved: -5%)

############ Column(cube) ############
name: cube
...

## ORC

In [31]:
val users = spark.read.orc("data/users.orc")
users.printSchema

root
 |-- name: string (nullable = true)
 |-- favorite_color: string (nullable = true)
 |-- favorite_numbers: array (nullable = true)
 |    |-- element: integer (containsNull = true)



users: DataFrame = [name: string, favorite_color: string ... 1 more field]

In [32]:
users.show

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          NULL|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [33]:
val example = spark.read.orc("data/example.orc")
example.printSchema

root
 |-- one: double (nullable = true)
 |-- two: string (nullable = true)
 |-- three: boolean (nullable = true)



example: DataFrame = [one: double, two: string ... 1 more field]

In [34]:
example.show

+----+---+-----+
| one|two|three|
+----+---+-----+
|-1.0|foo| true|
| NaN|bar|false|
| 2.5|baz| true|
+----+---+-----+



## Avro

In [35]:
import $ivy.`org.apache.spark::spark-avro:3.5.6`

import $ivy.$

In [36]:
val usersDF = spark.read.format("avro").load("data/users.avro")
usersDF.printSchema

root
 |-- name: string (nullable = true)
 |-- favorite_color: string (nullable = true)
 |-- favorite_numbers: array (nullable = true)
 |    |-- element: integer (containsNull = true)



usersDF: DataFrame = [name: string, favorite_color: string ... 1 more field]

In [37]:
usersDF.show

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          NULL|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [38]:
usersDF.select("name", "favorite_color").write.mode(SaveMode.Overwrite).format("avro").save("data/namesAndFavColors.avro")